# Ch.4 Real World Data Representation Using Tensors
---
실제 데이터를 사용해 신경망을 훈련시킨다.
먼저 이미지 데이터를 처리하는 방법에 대해 알아본다.

In [6]:
import torch
import imageio # 이미지 파일을 np array로 리턴하는 라이브러리

In [9]:
img_arr = imageio.imread('data/p1ch4/image-dog/bobby.jpg')
img_arr.shape # Channel X Height X Width

C:\Users\soso6\miniconda3\envs\Deeplearning with PyTorch\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  """Entry point for launching an IPython kernel.


(720, 1280, 3)

`permute` 메소드를 이용해 기존 차원의 배치를 알맞게 조정한다. 이때 `img`와 `out`은 같은 저장소를 참조한다.

In [18]:
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)
out.shape # Height X Width X Channel

torch.Size([3, 720, 1280])

`stack`을 사용해 미리 공간을 할당하고 새로운 이미지를 불러올 수 있다.

In [36]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [37]:
import os

data_dir = 'data/p1ch4/image-cats/'
filenames = [name for name in os.listdir(data_dir)
             if os.path.splitext(name)[-1] == '.png']
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3] # 투명도를 나타내는 채널이 붙어있는 사진도 있지만 여기서는 RGB만 사용한다.
    batch[i] = img_t

C:\Users\soso6\miniconda3\envs\Deeplearning with PyTorch\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  import sys


## 데이터 정규화
---

신경망은 0과 1 사이, 혹은 -1과 1 사이에서 훈련 성능이 가장 좋다.
데이터를 두 범위로 만들어주는 작업을 **정규화**라고 한다.
1. 자료형의 최댓값으로 나눠주는 방법
2. 데이터의 평균과 표준편차를 이용한 정규화 (= 통계 표준화)

이 외에도 모델을 강건(Robust)하게 만들기 위해 이미지를 회전, 크기 변환, 자르기 등의 작업을 할 수 있다.

In [41]:
# 자료형의 최댓값으로 나눠주는 방법
batch = batch.float()
batch /= 255.0 # uint8의 최댓값으로 나눈다.

In [44]:
# 데이터의 평균과 표준편차를 이용한 정규화 (= 통계 표준화)
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

## 용적(Volumetric) 데이터 다루기
---

CT 스캔 데이터는 흑백 이미지처럼 하나의 밀도 채널만 있다. 즉, 다른 색상 채널은 그대로 둔 채, 데이터를 저장하기도 한다.
따라서 데이터에는 이전에 살펴본 것처럼 세 개의 채널이 다 들어있다. 이 2차원 단면을 쌓아서 3차원 텐서를 만들면 대상의 3차원 해부도를 표현한 용적 데이터를 만들 수 있다.
근본적으로는 일반 이미지 데이터와 큰 차이가 없다. 채널 디멘션 뒤에 깊이(depth)차원을 가지기 때문에 *N* X *C* X *D* X *H* X *W*의 5차원 텐서가 된다.

![image](https://user-images.githubusercontent.com/76675506/188817297-280a7d62-b4c4-4554-94c4-bb6109083594.png)




In [45]:
dir_path = 'data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083'
vol_arr = imageio.volread(dir_path, 'DICOM') # DICOM: 의료용 디지털 영상(Digital Imaging and Communications in Medicine)
vol_arr.shape

Reading DICOM (examining files): 99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 99/99  (100.0%)


(99, 512, 512)

In [47]:
vol = torch.from_numpy(vol_arr).float()
vol = torch.unsqueeze(vol,0) # channel 차원 생성

vol.shape

torch.Size([1, 99, 512, 512])

## 테이블 데이터 다루기
---
테이블 형태로 저장된 데이터(CSV, 스프레드시트)는 각 행별로 독립적이고 열 별로 다른 데이터 타입을 가진 경우가 많다.
하지만 텐서는 모든 데이터가 부동소수점 형태로 저장되야 한다. 이를 처리하는 방법에 대해 다룬다.

In [48]:
import csv
import numpy as np
wine_path = 'data/p1ch4/tabular-wine/winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)

wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [50]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))

wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [51]:
wineq = torch.from_numpy((wineq_numpy))

wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [53]:
data = wineq[:, :-1]
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [62]:
target = wineq[:,-1].long()
target, target.shape

(tensor([6, 6, 6,  ..., 6, 7, 6]), torch.Size([4898]))

### 원핫 인코딩(One-got Encoding)

와인 데이터의 경우 target인 와인의 품질(quality) 값을 정수로 처리하는 것이 적절하다.
만약 점수가 품종 처럼 이산적일 경우에는 값 사이에 순서나 거리 개념이 없는 원 핫 인코딩이 좋다.
`scatter_` 메소드를 이용해 사용할 수 있다.
`scatter_`의 인자 값은 아래와 같다.
-

In [78]:
target_onehot = torch.zeros(target.shape[0], 10)

target_onehot.scatter_(1, target.unsqueeze(1), 1.0) # 각 행에서 target 레이블의 인덱스를 취한 다음, 열 인덱스 값으로 사용해서 해당 항목의 값을 1.0으로 지정한다.

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [84]:
target[0], target.unsqueeze(1)[0]

(tensor(6), tensor([6]))

In [94]:
data_mean = torch.mean(data, dim=0) # dim=0은 차원 0에 대해 축소 연산이 수행됨을 의미한다.
data_mean, data.shape

(tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
         1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01]),
 torch.Size([4898, 11]))

In [95]:
data_var = torch.var(data, dim=0)
data_var, data_var.shape

(tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
         1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00]),
 torch.Size([11]))

In [97]:
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized, data_normalized.shape

(tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
          -3.4915e-01, -1.3930e+00],
         [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
           1.3422e-03, -8.2419e-01],
         [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
          -4.3677e-01, -3.3663e-01],
         ...,
         [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
          -2.6153e-01, -9.0545e-01],
         [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
          -9.6251e-01,  1.8574e+00],
         [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
          -1.4882e+00,  1.0448e+00]]),
 torch.Size([4898, 11]))

In [98]:
bad_indexes = target <= 3
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [99]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [105]:
bad_data = data[target <= 3]
mid_data = data[target > 3 & (target < 7)]
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim = 0)
mid_mean = torch.mean(mid_data, dim = 0)
good_mean = torch.mean(good_data, dim = 0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'. format(i, *args))

 0 fixed acidity          7.60   6.85   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.33   0.33
 3 residual sugar         6.39   6.39   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.31  34.55
 6 total sulfur dioxide 170.60 138.36 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.19   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.51  11.42


In [106]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:,6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)

predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

In [107]:
actual_indexes = target > 5

actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [108]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()

n_matches, n_matches / n_predicted, n_matches / n_actual

(2018, 0.74000733406674, 0.6193984039287906)

# 시계열 데이터 다루기
---

시계열 데이터는 테이블의 행이 어떻게 배치되었는가가 영향을 끼친다.
시계열 데이터에 대해 알아보기 위해 공유 자전거 시스템의 시간대별 데이터셋을 살펴본다. 2차원의 데이터셋을 아래 그림처럼 3차원으로 바꿔본다.

![image](https://user-images.githubusercontent.com/76675506/188837954-ca1c62ce-83e1-498e-841b-96b20c094be2.png)

In [110]:
bikes_numpy = np.loadtxt(
    'data/p1ch4/bike-sharing-dataset/hour-fixed.csv',
    dtype=np.float32,
    delimiter=',',
    skiprows=1,
    converters={1: lambda x: float(x[8:10])} # 첫번째 열의 date string을 숫자로 변환
)
bikes = torch.from_numpy((bikes_numpy))
bikes, bikes.shape

(tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
          1.6000e+01],
         [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
          4.0000e+01],
         [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
          3.2000e+01],
         ...,
         [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
          9.0000e+01],
         [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
          6.1000e+01],
         [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
          4.9000e+01]]),
 torch.Size([17520, 17]))

이 신경망 모델은 대여 수나 하루 중 시각, 기온이나 날씨 상태 등의 값에 대해 연속으로 값을 볼 필요가 있다.
이를 크기가 C(channel=column)인 N(time axis)개의 병렬 시퀀스로 표현 가능하다.

### 시간 단위로 데이터 만들기

2년치 데이터셋을 쪼개서 일 단위로 만든다면 어떻게 될까?
길이가 *L*인 *C*개의 시퀀스를 가지는 *N*(샘플 수)개의 컬렉션을 얻게 된다. 즉 *N* X *C* X *L*로 만들어진다.
이 때 *C*에는 17개의 채널이, *L*은 24가 된다.

In [113]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

`view` 메소드를 이용해 저장 공간을 바꾸지 않고 즉, 연산 비용을 들이지 않고 텐서를 바꾼 것 같은 효과를 낸다.

In [112]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1]) # -1은 '남은 차원과 거기 들어있는 요소를 다 합친 것'을 의미하는 Placeholder로 사용한다.
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [114]:
# C개의 채널을 가진 하루를 L시간으로 나눈 N개의 연속값이라고 볼 때 이를 N X C X L 순서로 전치
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

'날씨 상태'의 데이터는 순서값이다. 1이 좋은 날씨고 4는 매우 안좋은 날씨를 가리킨다. 이를 카테고리로 볼 수 있고 연속값으로도 볼 수 있다.

In [116]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0],4)
first_day[:,9]

(tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2]),
 torch.Size([24, 4]))

In [117]:
weather_onehot.scatter_(
    dim=1,
    index=first_day[:,9].unsqueeze(1).long() -1, # 날씨는 1~4지만 색인은 0부터 시작하므로 1을 빼준다.
    value=1.0
)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [118]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [119]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4,
                                   daily_bikes.shape[2])
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [120]:
daily_weather_onehot.scatter_(
    1, daily_bikes[:, 9, :].long().unsqueeze(1) - 1,1.0
)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [121]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)

In [122]:
# temperature 열에 대한 정규화
daily_bikes[:, 9,:] = (daily_bikes[:, 9, :] - 1.0) / 3.0

In [123]:
# 값의 범위를 [0.0, 1.0]으로 매핑
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min) / (temp_max - temp_min))

In [125]:
# 통계적 표준화
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp)) / torch.std(temp))

시계열과 비슷한 형태로는 텍스트와 오디오가 있다.

# 텍스트 데이터 다루기
---
자연어 처리의 대표적인 모델로 **RNN**(**Recurrent Neural Network**)가 있다. 모델의 이전 출력과 현재의 입력을 섞는 식으로 반복하는 구조다.
최근에는 **트랜스포머**(**Transforemr**)로 과거의 정보를 포함하는 유연한 방법을 이용하는 모델도 있다.
이번 절의 목표는 텍스트를 신경망이 처리할 수 있도록 숫자로 이루어진 텐서로 바꾸는 것이다. 신경망이 텍스트를 다루는 방법은 아래와 같다.
1. 문자 단위로 하나의 문자를 처리하는 방법
2. 단어 단위로 개별 단어를 처리하는 방법

In [127]:
# 문자 단위
with open ('data/p1ch4/jane-austen/1342-0.txt', encoding='utf8') as f:
    text = f.read()

인코딩 종류를 무엇으로 정할지가 신경망의 성능에 영향을 끼칠 수 있다. 문자를 다 소문자로 바꾸거나, 문장 부호 등을 제거하는 방법으로 문자 종류를 줄일 수 있다.

In [128]:
lines = text.split('\n')
line = lines[200]
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

In [129]:
letter_t = torch.zeros((len(line), 128)) # ASCII 제한인 128로 하드코딩
letter_t.shape

torch.Size([70, 128])

In [130]:
for i, letter in enumerate(line.lower().strip()):
    letter_index = ord(letter) if ord(letter) < 128 else 0 # ASCII로 표현할 수 없는 문자는 버림
    letter_t[i][letter_index] = 1

단어 단위 인코딩도 같은 방식이다. 출현된 단어로 사전을 만들어 한 단어를 한 행으로 원 핫 인코딩한다.
사전에 포함되는 단어가 너무 많아 인코딩 벡터가 길어지면 실용성이 떨어지기 때문에 **임베딩**(**Embedding**)을 사용하기도 한다.

In [131]:
def clean_words(input_str):
    punctuation = '.,;:"!?”“_-'
    word_list = input_str.lower().replace('\n',' ').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

words_in_line = clean_words(line)
line, words_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

In [132]:
word_list = sorted((set(clean_words(text))))
word2index_dict = {word: i for (i, word) in enumerate(word_list)}

len(word2index_dict), word2index_dict['impossible']

(7261, 3394)

In [134]:
word_t = torch.zeros(len(words_in_line), len(word2index_dict))
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_t[i][word_index] = 1
    print('{:2} {:4} {}'. format(i, word_index, word))

print(word_t.shape)

 0 3394 impossible
 1 4305 mr
 2  813 bennet
 3 3394 impossible
 4 7078 when
 5 3315 i
 6  415 am
 7 4436 not
 8  239 acquainted
 9 7148 with
10 3215 him
torch.Size([11, 7261])


바이트 쌍 인코딩(Byte Pair Encoding)